<a href="https://colab.research.google.com/github/Alexandre77777/data_mining/blob/main/8.%20%D0%90%D0%BD%D1%81%D0%B0%D0%BC%D0%B1%D0%BB%D0%B5%D0%B2%D1%8B%D0%B5%20%D0%BC%D0%B5%D1%82%D0%BE%D0%B4%D1%8B%20%D0%BC%D0%B0%D1%88%D0%B8%D0%BD%D0%BD%D0%BE%D0%B3%D0%BE%20%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D1%8F/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0%20%E2%84%9611.%20%D0%90%D0%BD%D1%81%D0%B0%D0%BC%D0%B1%D0%BB%D0%B5%D0%B2%D1%8B%D0%B5_%D0%BC%D0%B5%D1%82%D0%BE%D0%B4%D1%8B_%D0%BC%D0%B0%D1%88%D0%B8%D0%BD%D0%BD%D0%BE%D0%B3%D0%BE_%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install catboost

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install xgboost

^C
Note: you may need to restart the kernel to use updated packages.


Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 99.8/99.8 MB 505.2 kB/s eta 0:00:00


In [3]:
%pip install lightgbm

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 1.3/1.3 MB 2.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


# Обучить модели машинного обучения для решения задачи классификации, используя следующие boosting-алгоритмы: CatBoost, Light GBM, XGBoost, AdaBoost. Сравнить их точность и быстродействие.

Датасет: https://www.kaggle.com/c/titanic/data

In [2]:
import pandas as pd
import numpy as np
from time import time
import plotly.express as px
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv(r'../data/titanic/train.csv')

In [4]:
df.shape

(891, 12)

In [5]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [7]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [8]:
df = df[df["Age"].notna() & df["Embarked"].notna()]

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  712 non-null    int64  
 1   Survived     712 non-null    int64  
 2   Pclass       712 non-null    int64  
 3   Name         712 non-null    object 
 4   Sex          712 non-null    object 
 5   Age          712 non-null    float64
 6   SibSp        712 non-null    int64  
 7   Parch        712 non-null    int64  
 8   Ticket       712 non-null    object 
 9   Fare         712 non-null    float64
 10  Cabin        183 non-null    object 
 11  Embarked     712 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 72.3+ KB


In [24]:
y = df['Survived']
X = df.drop(['Survived'], axis=1)

X = X[["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]]

In [25]:
import category_encoders as ce


encoder = ce.OrdinalEncoder(cols=['Sex', 'Embarked'])

X = encoder.fit_transform(X)

In [26]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    712 non-null    int64  
 1   Sex       712 non-null    int32  
 2   Age       712 non-null    float64
 3   SibSp     712 non-null    int64  
 4   Parch     712 non-null    int64  
 5   Fare      712 non-null    float64
 6   Embarked  712 non-null    int32  
dtypes: float64(2), int32(2), int64(3)
memory usage: 38.9 KB


In [27]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

accuracy = {}
speed = {}

## CatBoost


In [28]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(silent=True, allow_writing_files=False)

start = time()
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=42)
score = cross_val_score(model, X, y, scoring='f1', cv=cv, n_jobs=-1)

speed['CatBoost'] = np.round(time() - start, 3)
accuracy['CatBoost'] = (np.mean(score) * 100).round(3)


print(f"Mean F1 score: {accuracy['CatBoost']}")
print(f"STD: {np.std(score):.3f}")
print(f"Run Time: {speed['CatBoost']}s")

Mean F1 score: 75.994
STD: 0.038
Run Time: 3.774s


## Light GBM

In [29]:
from lightgbm import LGBMClassifier

model = LGBMClassifier()

start = time()
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=42)
score = cross_val_score(model, X, y, scoring='f1', cv=cv, n_jobs=-1)

speed['LGBM'] = np.round(time() - start, 3)
accuracy['LGBM'] = (np.mean(score) * 100).round(3)

print(f"Mean F1 score: {accuracy['LGBM']}")
print(f"STD: {np.std(score):.3f}")
print(f"Run Time: {speed['LGBM']}s")

Mean F1 score: 75.189
STD: 0.043
Run Time: 3.077s


## XGBoost

In [30]:
from xgboost import XGBClassifier

model = XGBClassifier()

start = time()
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=42)
score = cross_val_score(model, X, y, scoring='f1', cv=cv, n_jobs=-1)

speed['XGBoost'] = np.round(time() - start, 3)
accuracy['XGBoost'] = (np.mean(score) * 100).round(3)

print(f"Mean F1 score: {accuracy['XGBoost']}")
print(f"STD: {np.std(score):.3f}")
print(f"Run Time: {speed['XGBoost']}s")

Mean F1 score: 74.25
STD: 0.044
Run Time: 0.798s


## AdaBoost

In [31]:
from sklearn.ensemble import AdaBoostClassifier

model = AdaBoostClassifier()

start = time()
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=42)
score = cross_val_score(model, X, y, scoring='f1', cv=cv, n_jobs=-1)

speed['AdaBoost'] = np.round(time() - start, 3)
accuracy['AdaBoost'] = (np.mean(score) * 100).round(3)

print(f"Mean F1 score: {accuracy['AdaBoost']}")
print(f"STD: {np.std(score):.3f}")
print(f"Run Time: {speed['AdaBoost']}s")

Mean F1 score: 74.26
STD: 0.029
Run Time: 0.322s


## CatBoost vs LightGBM vs XGBoost vs AdaBoost

In [32]:
for algo, result in accuracy.items():
    print(f"{algo:{20}}: Score: {result}, Speed: {speed[algo]}")

CatBoost            : Score: 75.994, Speed: 3.774
LGBM                : Score: 75.189, Speed: 3.077
XGBoost             : Score: 74.25, Speed: 0.798
AdaBoost            : Score: 74.26, Speed: 0.322


# Обучить модели машинного обучения для решения задачи **регрессии**, используя следующие boosting-алгоритмы: [CatBoost](https://catboost.ai/en/docs/concepts/python-reference_catboostregressor), [Light GBM](https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMRegressor.html), [XGBoost](https://kirenz.github.io/regression/docs/xgboost-regression.html), [AdaBoost](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html). Сравнить их точность и быстродействие.

Датасет: https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_olivetti_faces.html

In [7]:
import pandas as pd
import numpy as np
from time import time
import plotly.express as px
import matplotlib.pyplot as plt

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_olivetti_faces


data = fetch_olivetti_faces()

X =  data.images.reshape(len(data.images), -1)
y = data.target

In [5]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score

accuracy = {}
speed = {}

## CatBoost

In [11]:
from catboost import CatBoostRegressor

model = CatBoostRegressor(random_state=33)

start = time()
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=42)
score = cross_val_score(model, X, y, scoring='r2', cv=cv, n_jobs=-1)


speed['CatBoost'] = np.round(time() - start, 3)
accuracy['CatBoost'] = (np.mean(score))

print(f"Mean r2 score: {accuracy['CatBoost']}")
print(f"STD: {np.std(score):.3f}")
print(f"Run Time: {speed['CatBoost']}s")

Mean r2 score: 0.6974031951656096
STD: 0.035
Run Time: 1301.78s


## LightGBM

In [12]:
from lightgbm import LGBMRegressor

model = LGBMRegressor(random_state=33)

start = time()
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=42)
score = cross_val_score(model, X, y, scoring='r2', cv=cv, n_jobs=-1)


speed['LGBM'] = np.round(time() - start, 3)
accuracy['LGBM'] = (np.mean(score))

print(f"Mean r2 score: {accuracy['LGBM']}")
print(f"STD: {np.std(score):.3f}")
print(f"Run Time: {speed['LGBM']}s")

Mean r2 score: 0.6479446503212206
STD: 0.044
Run Time: 21.525s


## XGBoost

In [10]:
from xgboost import XGBRegressor

model = XGBRegressor(random_state=33)

start = time()
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=42)
score = cross_val_score(model, X, y, scoring='r2', cv=cv, n_jobs=-1)


speed['XGBoost'] = np.round(time() - start, 3)
accuracy['XGBoost'] = (np.mean(score))

print(f"Mean r2 score: {accuracy['XGBoost']}")
print(f"STD: {np.std(score):.3f}")
print(f"Run Time: {speed['XGBoost']}s")

Mean r2 score: 0.5723590124912306
STD: 0.067
Run Time: 48.605s


## AdaBoost

In [9]:
from sklearn.ensemble import AdaBoostRegressor

model = AdaBoostRegressor(random_state=33)

start = time()
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=42)
score = cross_val_score(model, X, y, scoring='r2', cv=cv, n_jobs=-1)


speed['AdaBoost'] = np.round(time() - start, 3)
accuracy['AdaBoost'] = (np.mean(score))

print(f"Mean r2 score: {accuracy['AdaBoost']}")
print(f"STD: {np.std(score):.3f}")
print(f"Run Time: {speed['AdaBoost']}s")

Mean r2 score: 0.5897895552499297
STD: 0.047
Run Time: 21.952s


## CatBoost vs LightGBM vs XGBoost vs AdaBoost

In [13]:
for algo, result in accuracy.items():
    print(f"{algo:{20}}: Score: {result}, Speed: {speed[algo]}")

AdaBoost            : Score: 0.5897895552499297, Speed: 21.952
XGBoost             : Score: 0.5723590124912306, Speed: 48.605
CatBoost            : Score: 0.6974031951656096, Speed: 1301.78
LGBM                : Score: 0.6479446503212206, Speed: 21.525
